In [1]:
# 코드 작성일 : 24.7.4 -> 대표메뉴 추출 + 유사 메뉴 추출 
import os
from concurrent.futures import ThreadPoolExecutor

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.path.dirname(original_dir))))
exec(open('setup/default.py').read())

/Users/user/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [56]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

In [57]:
# 클렌징 
def filter_text(text):
    # '>'가 있는 경우 '>' 뒤의 텍스트 추출
    if '>' in text:
        text = text.replace('>,', '>') # 만약 <~>,이런식으로 붙어있으면 변경해주기
        text = text.split('>')[-1]
    # ','가 있는 경우 ',' 앞의 텍스트 추출
    if ',' in text:
        text = text.split(',')[0]
    # 특수문자 이후의 텍스트 제거
    text = re.sub(r'[(*&].*$', '', text).strip()
    # 공백 제거
    text = text.replace(' ', '')
    return text

# 데이터프레임에 함수 적용
menu_w_sold_out['first_menu'] = menu_w_sold_out['course'].apply(filter_text)


In [61]:
unique_first_menu = pd.DataFrame(menu_w_sold_out[['first_menu']].drop_duplicates()).reset_index(drop=True)
unique_first_menu.to_csv(
         '/Users/user/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/v2/unique_first_menu.csv',
         index= False
         )

In [78]:
similarity_df = pd.DataFrame()
for a_menu in tqdm(unique_first_menu.first_menu) :
    similarity_df_by_menu = unique_first_menu.copy()
    similarity_df_by_menu['sim_menu'] = a_menu
    similarity_df_by_menu['similarity'] = similarity_df_by_menu['first_menu'].apply(lambda x: fuzz.partial_ratio(x, a_menu))
    #similarity_df_by_menu = similarity_df_by_menu[similarity_df_by_menu.similarity>=70]
    similarity_df = pd.concat([similarity_df,similarity_df_by_menu])

100%|██████████| 663/663 [00:14<00:00, 44.51it/s]


In [80]:
similarity_df[similarity_df.first_menu == '투움바파스타'].sort_values(by='similarity',ascending =False)

,first_menu,sim_menu,similarity
0,투움바파스타,투움바파스타,100
0,투움바파스타,로제파스타,60
0,투움바파스타,바질페스토파스타,50
0,투움바파스타,달래알리오올리오파스타,50
0,투움바파스타,투움바리조또,50
...,...,...,...
0,투움바파스타,수프카레,0
0,투움바파스타,매콤비빔국수,0
0,투움바파스타,우렁강된장찌개,0
0,투움바파스타,철판오징어떡볶음,0


In [44]:
pd.to_csv(pd.DataFrame(menu_w_sold_out[['first_menu']].drop_duplicates()),''

0         투움바파스타
1            닭곰탕
2           육감쫄면
3         차돌된장찌개
4       매실청돈육불고기
          ...   
1674      반반만두전골
1675     불맛오징어덮밥
1685      묵은지닭갈비
1695        미니우동
1696      취나물버섯밥
Name: first_menu, Length: 664, dtype: object

In [6]:
first_menu_list = menu_w_sold_out.groupby(['first_menu'],dropna=False).size().to_frame('size').reset_index().sort_values(by='size',ascending=False)

In [7]:
# 띄어쓰기
spacing = Spacing()
sentences = first_menu_list.first_menu

def process_spacing(sentence):
    return spacing(sentence)

# ThreadPoolExecutor를 사용하여 동시에 5개의 작업을 수행합니다.
with ThreadPoolExecutor(max_workers=5) as executor:
    spaced_sentences = list(executor.map(process_spacing, sentences))

spaced_menu = pd.DataFrame({
    'first_menu': sentences,
    'first_menu_sp': spaced_sentences
})

In [9]:
#spaced_menu.to_csv('/Users/user/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/spaced_menu.csv',index=False)
spaced_menu=pd.read_csv('/Users/user/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/spaced_menu.csv')

In [10]:
menu_w_sold_out=pd.merge(menu_w_sold_out,spaced_menu, how='left',on='first_menu') 

In [11]:
first_menu_list = menu_w_sold_out.first_menu_sp.value_counts().to_frame().reset_index()

In [25]:
first_menu_list.sort_values(by='count',ascending=False).iloc[:10,:]

,first_menu_sp,count,similarity
0,뼈 없는 감자탕,21,18
1,닭곰탕,18,100
2,콩비지찌개,17,0
3,돼지국밥,16,0
4,마파 두부 덮밥,16,0
5,돼지 양념구이,15,0
6,설렁탕,15,33
7,가츠 동 돈부리,15,0
8,통마늘 돼 지불백,15,0
9,옛날 카레라이스,15,0


In [18]:
# 비교할 텍스트
target_text = '돼지 국밥'

# 유사도 계산하여 가장 유사한 메뉴 찾기
first_menu_list['similarity'] = first_menu_list['first_menu_sp'].apply(lambda x: fuzz.token_sort_ratio(x, target_text))
first_menu_list.sort_values(by='similarity',ascending=False).iloc[:10,:]

NameError: name 'first_menu_list' is not defined